In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [2]:
loader = TextLoader("all.txt")
documents = loader.load()

In [3]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=0)
docs = text_splitter.split_documents(documents)
embedding_function = SentenceTransformerEmbeddings(model_name="/data/usr/jy/asset/m3e-base")
len(docs)

/home/jy/anaconda3/envs/baichuan2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1054

In [7]:
import shutil
import os

# 数据库持久化目录路径
persist_directory = "./chroma_db"

# 检查目录是否存在
if os.path.exists(persist_directory):
    # 使用shutil.rmtree删除目录及其所有内容
    shutil.rmtree(persist_directory)
    print(f"数据库目录 {persist_directory} 已被删除。")
else:
    print(f"数据库目录 {persist_directory} 不存在。")

数据库目录 ./chroma_db 不存在。


In [23]:
db = Chroma.from_documents(docs, embedding_function, persist_directory="./chroma_demo_db")
db._collection.count()

4216

In [12]:
# 加载
db = Chroma(persist_directory="./chroma_demo_db", embedding_function=embedding_function)
document_paths = ["/data/usr/jy/Dataset/knowledge/毕业设计.txt", "/data/usr/jy/Dataset/knowledge/常用塑料知识.txt","/data/usr/jy/Dataset/knowledge/成型品不良的基本对策.txt"]
for doc_path in document_paths:
    loader = TextLoader(doc_path)
    documents = loader.load()
    # 使用RecursiveCharacterTextSplitter分割文档
    chunks = text_splitter.split_documents(documents)  # 假设split方法返回文档的分段列表
    # 对每个块生成向量并存储到数据库中
    for chunk in chunks:
        # 假设db.add_document(chunk)是添加单个块到数据库的方法
        db._collection_.method().add(chunk)

# 假设有一个finalize方法来完成数据库构建后的最终步骤
db.finalize()

AttributeError: 'Chroma' object has no attribute 'add_document'

In [26]:
# 删除
ids = [str(i) for i in range(1, len(docs) + 1)]
db._collection.delete(ids=[ids[-1]])

Delete of nonexisting embedding ID: 4
Delete of nonexisting embedding ID: 4


In [27]:
db._collection.count()

4216

In [25]:
query = "抛光树脂"
docs = db.similarity_search(query)
print(docs[0].page_content)

(1)聚酯树脂,机械强度高,成形方法容易,化学性能稳定。
(2)酚醛树脂,质脆,必须加入填料方能获得所要求的性能,价格低廉。
(3)环氧树脂,收缩率最小,机械性能高,耐酸、碱、盐和有机溶剂等化学药品的侵蚀,但抗冲击性能低,质脆,需加填充剂、稀释剂等来改善其性能。
(4)塑料钢,可作拉深模,其缺点是价格昂贵。
2)树脂模具的制作工艺
制作工艺过程如下:
(4)逆向工程技术制造模具零件
